# Data Fission
## Conducting Inference
We conduct inference on some vector $Y$ given a set of covariates $X$ and a known covariance matrix  $\Sigma = \sigma^2I_n$ as follows:
1. Decompose $y_i$ into $f(y_i) = y_i - Z_i$ and $g(y_i) = y_i + Z_i$ where $Z_i\sim\mathcal{N}(0,\sigma^2)$
2. Fit $f(y_i)$ using LASSO to select features, denoted as $M\subseteq [p]$ (tuning parameter $\lambda$ by 1 standard deviation rule)
3. Fit $g(y_i)$ by linear regression without regularization using only the selected features.
4. Construct CIs for the coeffecients trained in step 3. each at level $\alpha$ using Theorem 2.

### Theorem 2
Let 
$$\hat\beta(M) = \argmin_{\tilde\beta} = || g(Y) - X_M\tilde\beta||^2 = \left( X_M^\top X_M \right)^{-1} X_M^\top g(Y)$$

and for $\mu = \mathbb{E}[Y\mid X]\in\mathbb{R}^n$ (which is a fixed unknown quantity), 

$$\beta^*(M) = \argmin_{\tilde\beta} = \mathbb{E}\left[ || Y - X_M\tilde\beta ||^2 \right] = \left( X_M^\top X_M \right)^{-1} X_M^\top \mu.$$

Then, 

$$\hat\beta(M) \sim \mathcal{N} \left( \beta^*(M), \left( 1 +\tau^{-2} \right)\left( X_M^\top X_M\right)^{-1} X_M^\top \Sigma X_M \left( X_M^\top X_M\right)^{-1} \right)$$

Furthermore, we can form a $1-\alpha$ CI for the $k$th element of $\beta^*(M)$ as 

$$\hat\beta(M)\pm z_{\alpha/2}\sqrt{\left( 1 + \tau^{-2} \right) \left[ \left( X^\top_M X_M\right)^{-1} X_M^\top\Sigma X_M \left( X_M^\top X_M\right)^{-1} \right]_{kk} }$$

## Simulation Setup: 

"We choose $\sigma^2 = 1$ and generate $n = 16$ data points with $p = 20$ covariates. For the first 15 data points, we have an associated vector of covariates  $x_i\in\mathbb{R}^p$ generated from independent Gaussians. The last data point, which we denote $x_\text{lev}$, is generated in such a way as to ensure it is likely to bemore influential than the remaining observations due to having much larger leverage. We define 

$$x_\text{lev} = \gamma\left( \vert X_1 \vert_\infty, \dots, \vert X_p \vert_\infty \right)$$
  
where $X_k$ denotes the the kth column vector of themodel design matrix $X$ formed from the first 15 data points and $\gamma$ is a parameter that we will vary within these simulations that reflects the degree to which the last data point has higher leverage than the first set of data points. We then construct $y_i\sim\mathcal{N}\left( \beta^\top x_i,\sigma^2 \right)$. The parameter $\beta$ is nonzero for 4 features: $(\beta_{1}, \beta_{16}, \beta_{17}, \beta_{18}) = S_\Delta(1,1,-1,1)$ where $S_\Delta encodes signal strength." 

We use 500 repetitions and summarize performance as follows. For the selection stage, we compute the power (defined as $\frac{\vert j\in M:\beta_j\neq0\vert}{\vert j\in[p]:\beta_j\neq0\vert}$) and precision (defined as $\frac{\vert j\in M:\beta_j\neq0\vert}{\vert M\vert}$) of selecting features with a nonzero parameter. For inference, we use the false coverage rate (defined as $\frac{\vert k\in M:[\beta^*(M)]_k\not\in \text{CI}_k  \vert }{\max\{ \vert M \vert ,1 \}}$) where $\text{CI}_k$ is the CI for $[\beta^*(M)]_k. We also track the average CI length within the selected model.

In [563]:
import numpy as np
import pandas as pd
import tqdm

In [191]:
sigma_sq = 1
n = 15
p = 20
betas = np.zeros(p)

ones = [0,16,18]
betas[ones] = 1

neg_ones = [17]
betas[neg_ones] = -1


In [468]:
def generate_linear(n = n, p = p, betas = betas, add_influential = []):
    n_true  = n + len(add_influential)
    X = np.zeros((n_true, p))
    X_1_to_n = np.random.multivariate_normal(np.zeros(p), np.eye(p), n)
    X[:n,:] = X_1_to_n
    if len(add_influential) > 0:
        baseline = X_1_to_n.max(axis = 0)
        for i in range(len(add_influential)):
            X[(n - 1) + i,:] = baseline * add_influential[i]
    
    Y = np.random.normal(0, 1, n_true) + X @ betas
    sd = 1
    Sigma = np.eye(p)
    return X,Y, sd, Sigma

In [691]:
from sklearn.linear_model import ElasticNetCV, ElasticNet, LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.utils._testing import ignore_warnings
from sklearn.exceptions import ConvergenceWarning
from collections import namedtuple

methods = namedtuple('methods', ['masking', 'full', 'split', 'mysplit','loocv'])
method_results = namedtuple('method_results', ['CIs', 'projected', 'selected'])

def alpha1se_lasso(X, Y, cv = 10):
    model = ElasticNetCV(cv= cv, l1_ratio=1,max_iter=100000).fit(X,Y)
    mean_mse = np.mean(model.mse_path_, axis=1)
    alpha_min_index = np.argmin(mean_mse)
    mse_alpha_min = model.mse_path_[alpha_min_index,:]
    std_error = np.std(mse_alpha_min, ddof=1) / np.sqrt(model.mse_path_.shape[1])
    threshold = mean_mse[alpha_min_index] + std_error
    alpha_1se_indexes = np.where(mean_mse <= threshold)[0]
    alpha_1se = model.alphas_[alpha_1se_indexes[0]]
    return alpha_1se
    #return model.alpha_
    

def calculate_experiment_results(X, Y, selected, betas, Sigma, scale):
    if len(selected) == 0:
        return method_results(None, None, None)
    
    infer_model = LinearRegression().fit(X[:,selected], Y)
    
    ## 95% confidence intervals for the betas
    CIs = np.zeros((len(selected), 2))
    for i in range(len(selected)):
        X_i = X[:,selected[i]]
        X_not_i = np.delete(X[:,selected], i, axis = 1)
        beta_i = infer_model.coef_[i]
        sigma_sq_i = np.sum((Y - infer_model.predict(X[:,selected]))**2) / (len(Y) - len(selected))
        sigma_sq_not_i = np.sum((Y - infer_model.predict(X_not_i))**2) / (len(Y) - len(selected) - 1)
        t = beta_i / np.sqrt(sigma_sq_i * np.linalg.inv(X_i.T @ X_i)[0,0])
        t_crit = 2.228
        CIs[i,0] = beta_i - t_crit * np.sqrt(sigma_sq_i * np.linalg.inv(X_i.T @ X_i)[0,0])
        CIs[i,1] = beta_i + t_crit * np.sqrt(sigma_sq_i * np.linalg.inv(X_i.T @ X_i)[0,0])
    
    
    mask = np.ones(betas.shape, bool)
    mask[selected] = False
        
    projected = betas[selected]*scale + scale*betas[mask] @ Sigma[mask,:][:,selected] @ np.linalg.inv(Sigma[selected,:][:,selected])
    return method_results(CIs, projected, selected)

@ignore_warnings(category=ConvergenceWarning)
def experiment_linear(n = n, p = p, betas = betas, add_influential = []):
    scale = 1
    X, Y, sd, Sigma = generate_linear(n = n, p = p, betas = betas, add_influential = add_influential)
    n += len(add_influential)
    ################ Masking ########################################################
    sd_z = sd
    noise = np.random.normal(0, sd_z, n)
    g_Y = Y + noise
    h_Y = Y - noise
    masked_alpha_1se = alpha1se_lasso(X, g_Y)
    
    masked_lasso = ElasticNet(alpha=masked_alpha_1se, l1_ratio=1, max_iter=100000).fit(X,g_Y)
    
    masked_selected = np.where(masked_lasso.coef_ != 0)[0]
    masked_results = calculate_experiment_results(X, h_Y, masked_selected, betas, Sigma, scale)
    #################################################################################
    ################ Full ###########################################################
    full_alpha_1se = alpha1se_lasso(X, Y)
    full_lasso = ElasticNet(alpha=full_alpha_1se, l1_ratio=1, max_iter=100000).fit(X,Y)
    full_selected = np.where(full_lasso.coef_ != 0)[0]
    full_results = calculate_experiment_results(X, Y, full_selected, betas, Sigma, scale)
    #################################################################################
    ################ Split ##########################################################
    ## Split the data into training and testing, using binomial(1, 0.5) to decide
    ## which group each observation belongs to
    split = np.random.binomial(1, 0.5, n)
    if split.sum() < 10:
        split_results = method_results(None, None, None)
    else:
        X_test, X_train = [X[split == 0,:], X[split == 1,:]]
        Y_test, Y_train = [Y[split == 0], Y[split == 1]]
        if split.sum() < 10:
            try:
                split_alpha_1se = alpha1se_lasso(X_train, Y_train, cv = 8)
            except:
                split_alpha_1se = -10
        else:
            split_alpha_1se = alpha1se_lasso(X_train, Y_train)
        if split_alpha_1se == -10:
            split_results = method_results(None, None, None)
        else:
            split_lasso = ElasticNet(alpha=split_alpha_1se, l1_ratio=1, max_iter=100000).fit(X_train,Y_train)
            split_selected = np.where(split_lasso.coef_ != 0)[0]
            split_results = calculate_experiment_results(X_train, Y_train, split_selected, betas, Sigma, scale)
    #################################################################################
    ################ Split 2 ########################################################
    if split.sum() < 8:
        ## flip 0 and 1's in split
        split = 1 - split
    X_test, X_train = [X[split == 0,:], X[split == 1,:]]
    Y_test, Y_train = [Y[split == 0], Y[split == 1]]
    split2_alpha_1se = alpha1se_lasso(X_train, Y_train, cv = 8)
    split2_lasso = ElasticNet(alpha=split2_alpha_1se, l1_ratio=1, max_iter=100000).fit(X_train,Y_train)
    split2_selected = np.where(split2_lasso.coef_ != 0)[0]
    split2_results = calculate_experiment_results(X_train, Y_train, split2_selected, betas, Sigma, scale)
    ##############################################################################
    ################ LOOCV #######################################################
    ## Leave one out cross validation
    ## randomly pick a point to leave out from 0 to n-1
    left_out_index = np.random.choice(n)
    X_train = np.delete(X, left_out_index, axis = 0)
    Y_train = np.delete(Y, left_out_index)
    loocv_alpha_1se = alpha1se_lasso(X_train, Y_train)
    loocv_lasso = ElasticNet(alpha=loocv_alpha_1se, l1_ratio=1, max_iter=100000).fit(X_train,Y_train)
    loocv_selected = np.where(loocv_lasso.coef_ != 0)[0]
    loocv_results = calculate_experiment_results(X_train, Y_train, loocv_selected, betas, Sigma, scale)
    return methods(masked_results, full_results, split_results, split2_results, loocv_results)    
    
    


In [645]:
np.random.choice(16)

11

In [703]:
np.random.seed(200253431)
results_dict = {}
runs = 500
for lev in range(2, 7):
    results_dict[lev] = []
    ##
    pbar = tqdm.tqdm(range(runs))
    for i in pbar:
        with np.errstate(divide='ignore'):
            results_dict[lev].append(experiment_linear(add_influential = [lev]))
        ## update progress bar text
        pbar.set_description(f"Leverage {lev} - Run {i+1}")

Leverage 6 - Run 500: 100%|██████████| 500/500 [02:53<00:00,  2.88it/s]


In [708]:
plot_dict = {}
method_names = ['masking', 'full', 'split', 'mysplit', "loocv"]
CI_length_dict = {method: {lev: [] for lev in range(2,7)} for method in method_names}
for lev in range(2,7):
    for i in range(runs):
        for method in method_names:
            if getattr(results_dict[lev][i], method).CIs is None:
                continue
            CI_array = np.array(getattr(results_dict[lev][i], method).CIs)
            #try:
            #    CI_length = np.nanmean(CI_array[:,1] - CI_array[:,0])
            #except:
            #    print(CI_length)
            CI_length_dict[method][lev].extend((CI_array[:,1] - CI_array[:,0]))
            #for j in range(len(CI_length)):
            #    if not np.isnan(CI_length[j]):
            #        CI_length_dict[method][lev].append(CI_length[j])
            #if not np.isnan(CI_length):
            #    CI_length_dict[method][lev].append(CI_length)

In [696]:
results_dict[2][1].split.CIs

In [681]:
results_dict[3][4].mysplit.CIs

array([[-0.78572107,  1.65851177],
       [ 0.0802422 ,  3.09773544],
       [-1.33573541,  2.45152782],
       [-1.73953333,  2.13429813]])

In [709]:
#print(CI_length_dict["masking"][2].mean())
#print(CI_length_dict["full"][2].mean())
#print(CI_length_dict["split"][2].mean())
#print(CI_length_dict["mysplit"][2].mean())
#print(CI_length_dict["loocv"][2].mean())
for lev in range(2,7):
    print(f"Leverage {lev}")
    for method in method_names:
        print(f"{method}: {np.nanmean(np.array(CI_length_dict[method][lev]))}")
        ## print length of array
        print(f"Length: {len(CI_length_dict[method][lev])}")
    print("\n")
    

Leverage 2
masking: 4.395066238456812
Length: 2394
full: 1.3474945703179209
Length: 2889
split: 1.882967593814348
Length: 276
mysplit: 2.242569023570277
Length: 1287
loocv: 1.3365935120512207
Length: 2734


Leverage 3
masking: 4.133690409890507
Length: 2769
full: 1.774718331609656
Length: 3236
split: 2.1136942363312516
Length: 346
mysplit: 2.4519119409606436
Length: 1421
loocv: 1.8586667150520901
Length: 3004


Leverage 4
masking: 4.616462451328408
Length: 2931
full: 2.099303094558188
Length: 3227
split: 2.4046248129829073
Length: 383
mysplit: 3.2036274219436964
Length: 1385
loocv: 1.9057700352790232
Length: 3114


Leverage 5
masking: 3.817100800680296
Length: 2986
full: 1.9800338713905286
Length: 3217
split: 2.503251229299672
Length: 422
mysplit: 2.7641810964513227
Length: 1502
loocv: 1.7826174890972222
Length: 2876


Leverage 6
masking: 4.052516883006152
Length: 2993
full: 1.8156299839199117
Length: 3183
split: 2.3027788172648855
Length: 458
mysplit: 3.2253590266862058
Length: 1528
l